<a href="https://colab.research.google.com/github/lucianoon/1/blob/main/Lang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import logging
from datetime import datetime
from typing import Dict, List, Optional
from dataclasses import dataclass

from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.tools import Tool
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Configuração de logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@dataclass
class CustomerInfo:
    """Informações do cliente"""
    customer_id: str
    name: str
    email: str
    phone: str
    tier: str  # "bronze", "silver", "gold", "platinum"

class KnowledgeBase:
    """Base de conhecimento para consultas"""

    def __init__(self):
        self.vectorstore = None
        self.embeddings = OpenAIEmbeddings()

    def load_knowledge_base(self, file_path: str):
        """Carrega a base de conhecimento de um arquivo"""
        try:
            loader = TextLoader(file_path, encoding='utf-8')
            documents = loader.load()

            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200
            )
            texts = text_splitter.split_documents(documents)

            self.vectorstore = FAISS.from_documents(texts, self.embeddings)
            logger.info(f"Base de conhecimento carregada com {len(texts)} documentos")

        except Exception as e:
            logger.error(f"Erro ao carregar base de conhecimento: {e}")

    def search(self, query: str, k: int = 3) -> List[str]:
        """Busca informações na base de conhecimento"""
        if not self.vectorstore:
            return ["Base de conhecimento não carregada"]

        try:
            docs = self.vectorstore.similarity_search(query, k=k)
            return [doc.page_content for doc in docs]
        except Exception as e:
            logger.error(f"Erro na busca: {e}")
            return ["Erro ao buscar informações"]

class CustomerServiceAgent:
    """Agente de atendimento ao cliente com LangChain"""

    def __init__(self, openai_api_key: str):
        os.environ["OPENAI_API_KEY"] = openai_api_key

        # Inicializar componentes
        self.llm = ChatOpenAI(
            model="gpt-3.5-turbo",
            temperature=0.7,
            max_tokens=500
        )

        self.memory = ConversationBufferWindowMemory(
            k=10,
            memory_key="chat_history",
            return_messages=True
        )

        self.knowledge_base = KnowledgeBase()
        self.current_customer: Optional[CustomerInfo] = None

        # Configurar ferramentas
        self.tools = self._create_tools()

        # Configurar prompt do agente
        self.prompt = self._create_prompt()

        # Criar agente
        self.agent = create_openai_functions_agent(
            llm=self.llm,
            tools=self.tools,
            prompt=self.prompt
        )

        self.agent_executor = AgentExecutor(
            agent=self.agent,
            tools=self.tools,
            memory=self.memory,
            verbose=True,
            max_iterations=3
        )

    def _create_tools(self) -> List[Tool]:
        """Cria as ferramentas disponíveis para o agente"""

        def search_knowledge_base(query: str) -> str:
            """Busca informações na base de conhecimento"""
            results = self.knowledge_base.search(query)
            return "\n\n".join(results)

        def get_customer_info(customer_id: str) -> str:
            """Obtém informações do cliente"""
            # Simulação - em produção seria uma consulta ao banco de dados
            customers_db = {
                "12345": CustomerInfo("12345", "João Silva", "joao@email.com", "(11) 99999-9999", "gold"),
                "67890": CustomerInfo("67890", "Maria Santos", "maria@email.com", "(11) 88888-8888", "silver")
            }

            customer = customers_db.get(customer_id)
            if customer:
                self.current_customer = customer
                return f"Cliente: {customer.name}\nEmail: {customer.email}\nTelefone: {customer.phone}\nNível: {customer.tier}"
            return "Cliente não encontrado"

        def escalate_to_human(reason: str) -> str:
            """Escala o atendimento para um atendente humano"""
            logger.info(f"Escalação solicitada: {reason}")
            return f"Atendimento escalado para humano. Motivo: {reason}. Um atendente entrará em contato em breve."

        def check_order_status(order_id: str) -> str:
            """Consulta status do pedido"""
            # Simulação - em produção seria uma consulta ao sistema de pedidos
            orders_db = {
                "ORD001": {"status": "Em trânsito", "tracking": "BR123456789", "delivery_date": "2024-01-15"},
                "ORD002": {"status": "Entregue", "tracking": "BR987654321", "delivery_date": "2024-01-10"}
            }

            order = orders_db.get(order_id)
            if order:
                return f"Pedido {order_id}: {order['status']}\nCódigo de rastreamento: {order['tracking']}\nPrevisão de entrega: {order['delivery_date']}"
            return "Pedido não encontrado"

        return [
            Tool(
                name="search_knowledge_base",
                description="Busca informações na base de conhecimento da empresa",
                func=search_knowledge_base
            ),
            Tool(
                name="get_customer_info",
                description="Obtém informações do cliente usando o ID do cliente",
                func=get_customer_info
            ),
            Tool(
                name="escalate_to_human",
                description="Escala o atendimento para um atendente humano quando necessário",
                func=escalate_to_human
            ),
            Tool(
                name="check_order_status",
                description="Consulta o status de um pedido usando o ID do pedido",
                func=check_order_status
            )
        ]

    def _create_prompt(self) -> ChatPromptTemplate:
        """Cria o prompt do agente"""
        system_message = """Você é um assistente virtual de atendimento ao cliente altamente qualificado e empático.

INSTRUÇÕES:
- Seja sempre cordial, profissional e prestativo
- Use linguagem clara e acessível
- Mantenha o foco na resolução do problema do cliente
- Se não souber a resposta, use as ferramentas disponíveis ou escale para um humano
- Sempre confirme informações importantes com o cliente
- Personalize o atendimento baseado no perfil do cliente

QUANDO ESCALAR PARA HUMANO:
- Reclamações complexas ou sensíveis
- Solicitações de reembolso acima de R$ 500
- Problemas técnicos que não consegue resolver
- Cliente expressamente solicita falar com humano
- Situações que exigem tomada de decisão gerencial

FERRAMENTAS DISPONÍVEIS:
- search_knowledge_base: Para buscar informações sobre produtos, políticas, etc.
- get_ customer_info: Para obter dados do cliente
- escalate_to_human: Para transferir para atendente humano
- check_order_status: Para consultar status de pedidos

Hora atual: {current_time}
"""

        return ChatPromptTemplate.from_messages([
            ("system", system_message),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])

    def load_knowledge_base(self, file_path: str):
        """Carrega a base de conhecimento"""
        self.knowledge_base.load_knowledge_base(file_path)

    def chat(self, message: str) -> str:
        """Processa uma mensagem do cliente"""
        try:
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            # Adiciona contexto do cliente se disponível
            context = ""
            if self.current_customer:
                context = f"\nCliente atual: {self.current_customer.name} (Nível: {self.current_customer.tier})"

            response = self.agent_executor.invoke({
                "input": message + context,
                "current_time": current_time
            })

            return response["output"]

        except Exception as e:
            logger.error(f"Erro no processamento: {e}")
            return "Desculpe, ocorreu um erro interno. Vou transferir você para um atendente humano."

    def reset_conversation(self):
        """Reinicia a conversa"""
        self.memory.clear()
        self.current_customer = None
        logger.info("Conversa reiniciada")

# Exemplo de uso
def main():
    # Configurar chave da API OpenAI
    API_KEY = "sua-chave-openai-aqui"

    # Criar agente
    agent = CustomerServiceAgent(API_KEY)

    # Exemplo de base de conhecimento (criar arquivo knowledge_base.txt)
    knowledge_content = """
    POLÍTICA DE TROCA E DEVOLUÇÃO:
    - Prazo: 30 dias corridos a partir da data de recebimento
    - Produto deve estar em perfeitas condições
    - Embalagem original deve estar preservada
    - Taxa de reenvio por conta do cliente (exceto defeito de fabricação)

    FORMAS DE PAGAMENTO ACEITAS:
    - Cartão de crédito (Visa, Mastercard, Elo)
    - Cartão de débito
    - PIX
    - Boleto bancário
    - PayPal

    PRAZOS DE ENTREGA:
    - Região Sudeste: 3-5 dias úteis
    - Região Sul: 5-7 dias úteis
    - Região Nordeste: 7-10 dias úteis
    - Região Norte/Centro-Oeste: 10-15 dias úteis
    """

    # Salvar base de conhecimento em arquivo temporário
    with open("knowledge_base.txt", "w", encoding="utf-8") as f:
        f.write(knowledge_content)

    # Carregar base de conhecimento
    agent.load_knowledge_base("knowledge_base.txt")

    print("=== AGENTE DE ATENDIMENTO INICIADO ===")
    print("Digite 'sair' para encerrar ou 'reset' para reiniciar a conversa\n")

    while True:
        user_input = input("Cliente: ")

        if user_input.lower() == 'sair':
            break
        elif user_input.lower() == 'reset':
            agent.reset_conversation()
            print("Conversa reiniciada!\n")
            continue

        response = agent.chat(user_input)
        print(f"Atendente: {response}\n")

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'langchain_community'

In [2]:
from IPython import get_ipython
from IPython.display import display
# %%
import os
import logging
from datetime import datetime
from typing import Dict, List, Optional
from dataclasses import dataclass

# Update imports based on potential LangChain refactor
# from langchain.chat_models import ChatOpenAI # Old import
from langchain_community.chat_models import ChatOpenAI # New import

from langchain.memory import ConversationBufferWindowMemory
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.tools import Tool
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
# Update imports based on potential LangChain refactor
# from langchain.vectorstores import FAISS # Old import
# from langchain.embeddings import OpenAIEmbeddings # Old import
# from langchain.document_loaders import TextLoader # Old import
# from langchain.text_splitter import RecursiveCharacterTextSplitter # Old import
from langchain_community.vectorstores import FAISS # New import
from langchain_community.embeddings import OpenAIEmbeddings # New import
from langchain_community.document_loaders import TextLoader # New import
from langchain.text_splitter import RecursiveCharacterTextSplitter


# Configuração de logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@dataclass
class CustomerInfo:
    """Informações do cliente"""
    customer_id: str
    name: str
    email: str
    phone: str
    tier: str  # "bronze", "silver", "gold", "platinum"

class KnowledgeBase:
    """Base de conhecimento para consultas"""

    def __init__(self):
        self.vectorstore = None
        self.embeddings = OpenAIEmbeddings()

    def load_knowledge_base(self, file_path: str):
        """Carrega a base de conhecimento de um arquivo"""
        try:
            loader = TextLoader(file_path, encoding='utf-8')
            documents = loader.load()

            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200
            )
            texts = text_splitter.split_documents(documents)

            self.vectorstore = FAISS.from_documents(texts, self.embeddings)
            logger.info(f"Base de conhecimento carregada com {len(texts)} documentos")

        except Exception as e:
            logger.error(f"Erro ao carregar base de conhecimento: {e}")

    def search(self, query: str, k: int = 3) -> List[str]:
        """Busca informações na base de conhecimento"""
        if not self.vectorstore:
            return ["Base de conhecimento não carregada"]

        try:
            docs = self.vectorstore.similarity_search(query, k=k)
            return [doc.page_content for doc in docs]
        except Exception as e:
            logger.error(f"Erro na busca: {e}")
            return ["Erro ao buscar informações"]

class CustomerServiceAgent:
    """Agente de atendimento ao cliente com LangChain"""

    def __init__(self, openai_api_key: str):
        os.environ["OPENAI_API_KEY"] = openai_api_key

        # Inicializar componentes
        self.llm = ChatOpenAI(
            model="gpt-3.5-turbo",
            temperature=0.7,
            max_tokens=500
        )

        self.memory = ConversationBufferWindowMemory(
            k=10,
            memory_key="chat_history",
            return_messages=True
        )

        self.knowledge_base = KnowledgeBase()
        self.current_customer: Optional[CustomerInfo] = None

        # Configurar ferramentas
        self.tools = self._create_tools()

        # Configurar prompt do agente
        self.prompt = self._create_prompt()

        # Criar agente
        self.agent = create_openai_functions_agent(
            llm=self.llm,
            tools=self.tools,
            prompt=self.prompt
        )

        self.agent_executor = AgentExecutor(
            agent=self.agent,
            tools=self.tools,
            memory=self.memory,
            verbose=True,
            max_iterations=3
        )

    def _create_tools(self) -> List[Tool]:
        """Cria as ferramentas disponíveis para o agente"""

        def search_knowledge_base(query: str) -> str:
            """Busca informações na base de conhecimento"""
            results = self.knowledge_base.search(query)
            return "\n\n".join(results)

        def get_customer_info(customer_id: str) -> str:
            """Obtém informações do cliente"""
            # Simulação - em produção seria uma consulta ao banco de dados
            customers_db = {
                "12345": CustomerInfo("12345", "João Silva", "joao@email.com", "(11) 99999-9999", "gold"),
                "67890": CustomerInfo("67890", "Maria Santos", "maria@email.com", "(11) 88888-8888", "silver")
            }

            customer = customers_db.get(customer_id)
            if customer:
                self.current_customer = customer
                return f"Cliente: {customer.name}\nEmail: {customer.email}\nTelefone: {customer.phone}\nNível: {customer.tier}"
            return "Cliente não encontrado"

        def escalate_to_human(reason: str) -> str:
            """Escala o atendimento para um atendente humano"""
            logger.info(f"Escalação solicitada: {reason}")
            return f"Atendimento escalado para humano. Motivo: {reason}. Um atendente entrará em contato em breve."

        def check_order_status(order_id: str) -> str:
            """Consulta status do pedido"""
            # Simulação - em produção seria uma consulta ao sistema de pedidos
            orders_db = {
                "ORD001": {"status": "Em trânsito", "tracking": "BR123456789", "delivery_date": "2024-01-15"},
                "ORD002": {"status": "Entregue", "tracking": "BR987654321", "delivery_date": "2024-01-10"}
            }

            order = orders_db.get(order_id)
            if order:
                return f"Pedido {order_id}: {order['status']}\nCódigo de rastreamento: {order['tracking']}\nPrevisão de entrega: {order['delivery_date']}"
            return "Pedido não encontrado"

        return [
            Tool(
                name="search_knowledge_base",
                description="Busca informações na base de conhecimento da empresa",
                func=search_knowledge_base
            ),
            Tool(
                name="get_customer_info",
                description="Obtém informações do cliente usando o ID do cliente",
                func=get_customer_info
            ),
            Tool(
                name="escalate_to_human",
                description="Escala o atendimento para um atendente humano quando necessário",
                func=escalate_to_human
            ),
            Tool(
                name="check_order_status",
                description="Consulta o status de um pedido usando o ID do pedido",
                func=check_order_status
            )
        ]

    def _create_prompt(self) -> ChatPromptTemplate:
        """Cria o prompt do agente"""
        system_message = """Você é um assistente virtual de atendimento ao cliente altamente qualificado e empático.

INSTRUÇÕES:
- Seja sempre cordial, profissional e prestativo
- Use linguagem clara e acessível
- Mantenha o foco na resolução do problema do cliente
- Se não souber a resposta, use as ferramentas disponíveis ou escale para um humano
- Sempre confirme informações importantes com o cliente
- Personalize o atendimento baseado no perfil do cliente

QUANDO ESCALAR PARA HUMANO:
- Reclamações complexas ou sensíveis
- Solicitações de reembolso acima de R$ 500
- Problemas técnicos que não consegue resolver
- Cliente expressamente solicita falar com humano
- Situações que exigem tomada de decisão gerencial

FERRAMENTAS DISPONÍVEIS:
- search_knowledge_base: Para buscar informações sobre produtos, políticas, etc.
- get_ customer_info: Para obter dados do cliente
- escalate_to_human: Para transferir para atendente humano
- check_order_status: Para consultar status de pedidos

Hora atual: {current_time}
"""

        return ChatPromptTemplate.from_messages([
            ("system", system_message),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])

    def load_knowledge_base(self, file_path: str):
        """Carrega a base de conhecimento"""
        self.knowledge_base.load_knowledge_base(file_path)

    def chat(self, message: str) -> str:
        """Processa uma mensagem do cliente"""
        try:
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            # Adiciona contexto do cliente se disponível
            context = ""
            if self.current_customer:
                context = f"\nCliente atual: {self.current_customer.name} (Nível: {self.current_customer.tier})"

            response = self.agent_executor.invoke({
                "input": message + context,
                "current_time": current_time
            })

            return response["output"]

        except Exception as e:
            logger.error(f"Erro no processamento: {e}")
            return "Desculpe, ocorreu um erro interno. Vou transferir você para um atendente humano."

    def reset_conversation(self):
        """Reinicia a conversa"""
        self.memory.clear()
        self.current_customer = None
        logger.info("Conversa reiniciada")

# Exemplo de uso
def main():
    # Configurar chave da API OpenAI
    API_KEY = "sua-chave-openai-aqui"

    # Criar agente
    agent = CustomerServiceAgent(API_KEY)

    # Exemplo de base de conhecimento (criar arquivo knowledge_base.txt)
    knowledge_content = """
    POLÍTICA DE TROCA E DEVOLUÇÃO:
    - Prazo: 30 dias corridos a partir da data de recebimento
    - Produto deve estar em perfeitas condições
    - Embalagem original deve estar preservada
    - Taxa de reenvio por conta do cliente (exceto defeito de fabricação)

    FORMAS DE PAGAMENTO ACEITAS:
    - Cartão de crédito (Visa, Mastercard, Elo)
    - Cartão de débito
    - PIX
    - Boleto bancário
    - PayPal

    PRAZOS DE ENTREGA:
    - Região Sudeste: 3-5 dias úteis
    - Região Sul: 5-7 dias úteis
    - Região Nordeste: 7-10 dias úteis
    - Região Norte/Centro-Oeste: 10-15 dias úteis
    """

    # Salvar base de conhecimento em arquivo temporário
    with open("knowledge_base.txt", "w", encoding="utf-8") as f:
        f.write(knowledge_content)

    # Carregar base de conhecimento
    agent.load_knowledge_base("knowledge_base.txt")

    print("=== AGENTE DE ATENDIMENTO INICIADO ===")
    print("Digite 'sair' para encerrar ou 'reset' para reiniciar a conversa\n")

    while True:
        user_input = input("Cliente: ")

        if user_input.lower() == 'sair':
            break
        elif user_input.lower() == 'reset':
            agent.reset_conversation()
            print("Conversa reiniciada!\n")
            continue

        response = agent.chat(user_input)
        print(f"Atendente: {response}\n")

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'langchain_community'

In [ ]:
!pip install langchain-community
!pip install langchain-openai # Ensure OpenAI integration is also available in community package

import os
import logging
from datetime import datetime
from typing import Dict, List, Optional
from dataclasses import dataclass

# Update imports based on potential LangChain refactor
# from langchain.chat_models import ChatOpenAI # Old import
from langchain_community.chat_models import ChatOpenAI # New import

from langchain.memory import ConversationBufferWindowMemory
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.tools import Tool
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
# Update imports based on potential LangChain refactor
# from langchain.vectorstores import FAISS # Old import
# from langchain.embeddings import OpenAIEmbeddings # Old import
# from langchain.document_loaders import TextLoader # Old import
# from langchain.text_splitter import RecursiveCharacterTextSplitter # Old import
from langchain_community.vectorstores import FAISS # New import
from langchain_community.embeddings import OpenAIEmbeddings # New import
from langchain_community.document_loaders import TextLoader # New import
from langchain.text_splitter import RecursiveCharacterTextSplitter


# Configuração de logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@dataclass
class CustomerInfo:
    """Informações do cliente"""
    customer_id: str
    name: str
    email: str
    phone: str
    tier: str  # "bronze", "silver", "gold", "platinum"

class KnowledgeBase:
    """Base de conhecimento para consultas"""

    def __init__(self):
        self.vectorstore = None
        # Ensure OpenAIEmbeddings is imported from langchain_community
        self.embeddings = OpenAIEmbeddings()

    def load_knowledge_base(self, file_path: str):
        """Carrega a base de conhecimento de um arquivo"""
        try:
            # Ensure TextLoader is imported from langchain_community
            loader = TextLoader(file_path, encoding='utf-8')
            documents = loader.load()

            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200
            )
            texts = text_splitter.split_documents(documents)

            # Ensure FAISS is imported from langchain_community
            self.vectorstore = FAISS.from_documents(texts, self.embeddings)
            logger.info(f"Base de conhecimento carregada com {len(texts)} documentos")

        except Exception as e:
            logger.error(f"Erro ao carregar base de conhecimento: {e}")

    def search(self, query: str, k: int = 3) -> List[str]:
        """Busca informações na base de conhecimento"""
        if not self.vectorstore:
            return ["Base de conhecimento não carregada"]

        try:
            # similarity_search is a method of the vectorstore object,
            # which was created using FAISS from langchain_community, so this should work.
            docs = self.vectorstore.similarity_search(query, k=k)
            return [doc.page_content for doc in docs]
        except Exception as e:
            logger.error(f"Erro na busca: {e}")
            return ["Erro ao buscar informações"]

class CustomerServiceAgent:
    """Agente de atendimento ao cliente com LangChain"""

    def __init__(self, openai_api_key: str):
        os.environ["OPENAI_API_KEY"] = openai_api_key

        # Inicializar componentes
        # Ensure ChatOpenAI is imported from langchain_community
        self.llm = ChatOpenAI(
            model="gpt-3.5-turbo",
            temperature=0.7,
            max_tokens=500
        )

        # ConversationBufferWindowMemory is still in langchain.memory
        self.memory = ConversationBufferWindowMemory(
            k=10,
            memory_key="chat_history",
            return_messages=True
        )

        self.knowledge_base = KnowledgeBase()
        self.current_customer: Optional[CustomerInfo] = None

        # Configurar ferramentas
        self.tools = self._create_tools()

        # Configurar prompt do agente
        self.prompt = self._create_prompt()

        # Criar agente
        # create_openai_functions_agent is still in langchain.agents
        self.agent = create_openai_functions_agent(
            llm=self.llm,
            tools=self.tools,
            prompt=self.prompt
        )

        # AgentExecutor is still in langchain.agents
        self.agent_executor = AgentExecutor(
            agent=self.agent,
            tools=self.tools,
            memory=self.memory,
            verbose=True,
            max_iterations=3
        )

    def _create_tools(self) -> List[Tool]:
        """Cria as ferramentas disponíveis para o agente"""

        def search_knowledge_base(query: str) -> str:
            """Busca informações na base de conhecimento"""
            results = self.knowledge_base.search(query)
            return "\n\n".join(results)

        def get_customer_info(customer_id: str) -> str:
            """Obtém informações do cliente"""
            # Simulação - em produção seria uma consulta ao banco de dados
            customers_db = {
                "12345": CustomerInfo("12345", "João Silva", "joao@email.com", "(11) 99999-9999", "gold"),
                "67890": CustomerInfo("67890", "Maria Santos", "maria@email.com", "(11) 88888-8888", "silver")
            }

            customer = customers_db.get(customer_id)
            if customer:
                self.current_customer = customer
                return f"Cliente: {customer.name}\nEmail: {customer.email}\nTelefone: {customer.phone}\nNível: {customer.tier}"
            return "Cliente não encontrado"

        def escalate_to_human(reason: str) -> str:
            """Escala o atendimento para um atendente humano"""
            logger.info(f"Escalação solicitada: {reason}")
            return f"Atendimento escalado para humano. Motivo: {reason}. Um atendente entrará em contato em breve."

        def check_order_status(order_id: str) -> str:
            """Consulta status do pedido"""
            # Simulação - em produção seria uma consulta ao sistema de pedidos
            orders_db = {
                "ORD001": {"status": "Em trânsito", "tracking": "BR123456789", "delivery_date": "2024-01-15"},
                "ORD002": {"status": "Entregue", "tracking": "BR987654321", "delivery_date": "2024-01-10"}
            }

            order = orders_db.get(order_id)
            if order:
                return f"Pedido {order_id}: {order['status']}\nCódigo de rastreamento: {order['tracking']}\nPrevisão de entrega: {order['delivery_date']}"
            return "Pedido não encontrado"

        # Tool is still in langchain.tools
        return [
            Tool(
                name="search_knowledge_base",
                description="Busca informações na base de conhecimento da empresa",
                func=search_knowledge_base
            ),
            Tool(
                name="get_customer_info",
                description="Obtém informações do cliente usando o ID do cliente",
                func=get_customer_info
            ),
            Tool(
                name="escalate_to_human",
                description="Escala o atendimento para um atendente humano quando necessário",
                func=escalate_to_human
            ),
            Tool(
                name="check_order_status",
                description="Consulta o status de um pedido usando o ID do pedido",
                func=check_order_status
            )
        ]

    def _create_prompt(self) -> ChatPromptTemplate:
        """Cria o prompt do agente"""
        system_message = """Você é um assistente virtual de atendimento ao cliente altamente qualificado e empático.

INSTRUÇÕES:
- Seja sempre cordial, profissional e prestativo
- Use linguagem clara e acessível
- Mantenha o foco na resolução do problema do cliente
- Se não souber a resposta, use as ferramentas disponíveis ou escale para um humano
- Sempre confirme informações importantes com o cliente
- Personalize o atendimento baseado no perfil do cliente

QUANDO ESCALAR PARA HUMANO:
- Reclamações complexas ou sensíveis
- Solicitações de reembolso acima de R$ 500
- Problemas técnicos que não consegue resolver
- Cliente expressamente solicita falar com humano
- Situações que exigem tomada de decisão gerencial

FERRAMENTAS DISPONÍVEIS:
- search_knowledge_base: Para buscar informações sobre produtos, políticas, etc.
- get_ customer_info: Para obter dados do cliente
- escalate_to_human: Para transferir para atendente humano
- check_order_status: Para consultar status de pedidos

Hora atual: {current_time}
"""
        # ChatPromptTemplate and MessagesPlaceholder are still in langchain.prompts
        return ChatPromptTemplate.from_messages([
            ("system", system_message),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])

    def load_knowledge_base(self, file_path: str):
        """Carrega a base de conhecimento"""
        self.knowledge_base.load_knowledge_base(file_path)

    def chat(self, message: str) -> str:
        """Processa uma mensagem do cliente"""
        try:
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            # Adiciona contexto do cliente se disponível
            context = ""
            if self.current_customer:
                context = f"\nCliente atual: {self.current_customer.name} (Nível: {self.current_customer.tier})"

            # The invoke method call looks correct.
            response = self.agent_executor.invoke({
                "input": message + context,
                "current_time": current_time
            })

            return response["output"]

        except Exception as e:
            logger.error(f"Erro no processamento: {e}")
            return "Desculpe, ocorreu um erro interno. Vou transferir você para um atendente humano."

    def reset_conversation(self):
        """Reinicia a conversa"""
        self.memory.clear()
        self.current_customer = None
        logger.info("Conversa reiniciada")

# Exemplo de uso
def main():
    # Configurar chave da API OpenAI
    # Replace "sua-chave-openai-aqui" with your actual OpenAI API key
    # Or better, set it as an environment variable before running the script
    # os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
    API_KEY = os.environ.get("OPENAI_API_KEY", "sua-chave-openai-aqui")
    if API_KEY == "sua-chave-openai-aqui":
        print("ATENÇÃO: Substitua 'sua-chave-openai-aqui' pela sua chave real da API OpenAI ou defina a variável de ambiente OPENAI_API_KEY.")
        # You might want to exit or handle this differently in a real application
        # sys.exit("Chave da API OpenAI não configurada.")


    # Criar agente
    agent = CustomerServiceAgent(API_KEY)

    # Exemplo de base de conhecimento (criar arquivo knowledge_base.txt)
    knowledge_content = """
    POLÍTICA DE TROCA E DEVOLUÇÃO:
    - Prazo: 30 dias corridos a partir da data de recebimento
    - Produto deve estar em perfeitas condições
    - Embalagem original deve estar preservada
    - Taxa de reenvio por conta do cliente (exceto defeito de fabricação)

    FORMAS DE PAGAMENTO ACEITAS:
    - Cartão de crédito (Visa, Mastercard, Elo)
    - Cartão de débito
    - PIX
    - Boleto bancário
    - PayPal

    PRAZOS DE ENTREGA:
    - Região Sudeste: 3-5 dias úteis
    - Região Sul: 5-7 dias úteis
    - Região Nordeste: 7-10 dias úteis
    - Região Norte/Centro-Oeste: 10-15 dias úteis
    """

    # Salvar base de conhecimento em arquivo temporário
    with open("knowledge_base.txt", "w", encoding="utf-8") as f:
        f.write(knowledge_content)

    # Carregar base de conhecimento
    agent.load_knowledge_base("knowledge_base.txt")

    print("=== AGENTE DE ATENDIMENTO INICIADO ===")
    print("Digite 'sair' para encerrar ou 'reset' para reiniciar a conversa\n")

    while True:
        user_input = input("Cliente: ")

        if user_input.lower() == 'sair':
            break
        elif user_input.lower() == 'reset':
            agent.reset_conversation()
            print("Conversa reiniciada!\n")
            continue

        response = agent.chat(user_input)
        print(f"Atendente: {response}\n")

if __name__ == "__main__":
    main()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.44
    Uninstalling langsmith-0.3.44:
      Successfully uninstalled langsmith-0.3.44
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.63
    Uninstalling langchain-core-0.3.63:
      Successfully uninstalled langchain-core-0.3.63
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 3.5 MB/s eta 0:00:00
ATENÇÃO: Substitua 'sua-chave-openai-aqui' pela sua chave real da API OpenAI ou defina a variável de ambiente OPENAI_API_KEY.


<ipython-input-3-1242874988>:93: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  self.llm = ChatOpenAI(
<ipython-input-3-1242874988>:100: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  self.memory = ConversationBufferWindowMemory(
<ipython-input-3-1242874988>:49: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings`

=== AGENTE DE ATENDIMENTO INICIADO ===
Digite 'sair' para encerrar ou 'reset' para reiniciar a conversa

Cliente: qual a politica da empresa


ERROR:__main__:Erro no processamento: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sua-chav*********aqui. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}




> Entering new AgentExecutor chain...
Atendente: Desculpe, ocorreu um erro interno. Vou transferir você para um atendente humano.

Cliente: meu id é or14


ERROR:__main__:Erro no processamento: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sua-chav*********aqui. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}




> Entering new AgentExecutor chain...
Atendente: Desculpe, ocorreu um erro interno. Vou transferir você para um atendente humano.

